In [2]:
# !pip install langchain==0.1.19
# !pip install langchain-openai==0.1.6
# !pip install langchain-community==0.0.38
# %pip install -qU pypdf

In [10]:
from google.colab import drive
from getpass import getpass
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import Optional


In [ ]:
file_path = r"...path_to_pdf"

In [5]:
OPENAPI_KEY = getpass()
os.environ['OPENAI_API_KEY'] = OPENAPI_KEY

··········


# Study Assistant

## 1. Read the uploaded study material (only pdf)
## 2. Generate Summary
## 3. Create Quiz Questions with answer options

## Load input file

In [6]:
loader = PyPDFLoader(file_path)
input_file = loader.lazy_load()
content = " ".join([page.page_content for page in input_file])

## Output Format

In [7]:
class MainPoints(BaseModel):
  point_number: int = Field(description="Point Number")
  point_text: str = Field(description="Main Points of the summary")

class Option(BaseModel):
    option_id: str = Field(description="A,B,C,D")
    answer_text: str = Field(description="option text")

class Question(BaseModel):
    question_no: int = Field(description="Question Number")
    question: str = Field(description="Quiz Question")
    options: list[Option]
    difficulty: str = Field(description = "The difficulty level of the question. Easy, Medium or Hard")
    answer : str = Field(description="Correct Answer")


class MCQ(BaseModel):
    topic: str = Field(description="Topic of the study material")
    summary: str = Field(description="Summary of the study material")
    main_points: list[MainPoints]
    questions: list[Question]

parser = PydanticOutputParser(pydantic_object=MCQ)
parser

PydanticOutputParser(pydantic_object=<class '__main__.MCQ'>)

In [11]:
prompt_text = """You are a brilliant study assistant, you read through the content provided by the user and summarize it.
The summary should contain
- numbered bullet points for each main point with a crisp and concise explanation.

Once you are done with the summarizing, generate a quiz with {no_of_questions} number of multiple choice questions from the summarized content ,
each with its own set of answer options. The questions should
- be from the study content only
- range from easy, medium to hard levels
- test the user's knowledge of the content
- should not be the same option number for all the questions
Format Instructions:{format_instructions}
content : {content}
no_of_questions : {no_of_questions} """

prompt = PromptTemplate(
    template=prompt_text,
    input_variables=["content","no_of_questions"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

#prompt

In [12]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [13]:
chain = ( prompt | model | parser)

In [14]:
response = chain.invoke({"content": content,"no_of_questions":5})

In [15]:
print(f"Topic : {response.topic}\n")
print(f"Summary : {response.summary}\n")
print(f"Highlights : \n")
for point in response.main_points:
  print(f"{point.point_number}. {point.point_text}")
for question in response.questions:
  print(f"\n{question.question_no}. {question.question} \n")
  print(f"Difficulty level : {question.difficulty} \n")
  for option in question.options:
    print(f"{option.option_id}. {option.answer_text}")

  print(f"\n Answer : {question.answer}\n")

Topic : Prompt Engineering in AI

Summary : Prompt engineering is a practice within natural language processing (NLP) in artificial intelligence, where text is used to describe the task the AI should perform. Prompts are detailed descriptions of the desired output from an AI model. Examples of prompts include text prompts, code prompts, and image prompts. To engineer effective AI prompts, one should consider role-playing, clarity, specification, and consistency. Standard prompt patterns include user-model interaction, few-shot prompting, and question-and-answer pattern. Advanced prompting techniques include zero-shot prompting, few-shot prompting, and chain-of-thought (CoT). Pitfalls to avoid when creating prompts include information overload, open-ended questions, and poor use of constraints.

Highlights : 

1. Prompt engineering involves using text to describe the task an AI should perform.
2. Prompts are detailed descriptions of the desired output from an AI model.
3. To engineer ef